In [ ]:
! pip install gdown accelerate bitsandbytes datasets

In [ ]:
! gdown 1m3foUPaSmj4omSEAzi4cJV_Sw1hAfKRn

In [ ]:
! gdown 1Imqc0XrkBChJ4z31MpwGF8eoz--Oge_9
! unzip interiorverse120.zip

In [ ]:
! gdown 1nL1U_UirQTKfjMO-Sk90kLhWBo1LYG2N

Downloading...
From: https://drive.google.com/uc?id=1nL1U_UirQTKfjMO-Sk90kLhWBo1LYG2N
To: /kaggle/working/image_captions.json
100%|██████████████████████████████████████| 24.3M/24.3M [00:00<00:00, 91.2MB/s]


In [ ]:
import os
from PIL import Image
import json
import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration

In [ ]:
model_id = "llava-hf/llava-1.5-7b-hf"
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_4bit=True,
)#.to(0)

In [ ]:
image_folder = "/kaggle/working/im_exr_120_jpg"
batch_size = 8
processor = AutoProcessor.from_pretrained(model_id)

In [ ]:
def generate_captions_for_batch(image_paths):
    batch_prompts = []
    batch_images = []

    for image_path in image_paths:
        raw_image = Image.open(image_path)
        batch_images.append(raw_image)

        prompt = "USER: <image>\nWhat is happening in the image?\nASSISTANT:"
        batch_prompts.append(prompt)

    inputs = processor(batch_prompts, batch_images, return_tensors='pt', padding=True).to(0, torch.float16)

    outputs = model.generate(**inputs, max_new_tokens=200, do_sample=False)

    captions = []
    for output in outputs:
        decoded_output = processor.decode(output[2:], skip_special_tokens=True)

        relevant_text = decoded_output.split("\nASSISTANT:")[1].strip()
        captions.append(relevant_text)

    return captions

In [ ]:
image_captions = {}

all_image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
for i in range(0, len(all_image_paths), batch_size):
    batch_paths = all_image_paths[i:i+batch_size]
    batch_captions = generate_captions_for_batch(batch_paths)

    for path, caption in zip(batch_paths, batch_captions):
        image_captions[os.path.basename(path)] = caption


In [ ]:
with open('image_captions.json', 'w') as file:
    json.dump(image_captions, file, indent=4)


In [ ]:
import json
import torch
from transformers import pipeline

2024-03-05 10:35:05.500532: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 10:35:05.500584: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 10:35:05.502060: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
f = open('/kaggle/working/image_captions.json')
#f = open('/kaggle/input/ikea-json/image_captions-ikea.json')
data = json.load(f)

In [ ]:
pipe = pipeline(
    "text-generation",
    model="HuggingFaceH4/zephyr-7b-gemma-v0.1",
    device_map="auto",
    torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
prompt = "Given a detailed description of a room's interior, identify all the distinct objects mentioned in the description. List these objects separated by commas, focusing solely on the items present in the room without including any descriptive or contextual information. For example, if the description mentions 'a cozy bedroom with a neatly made bed under a window, a dresser, and a potted plant', you should return 'bed, window, dresser, potted plant'."
description = ""

filenames = []
all_messages = []

In [ ]:
messages = [
    {
        "role": "system",
        "content": "",
    },
    {"role": "user", "content": None},
]

In [ ]:
for k in data.keys():
    filenames.append(k)
    messages[1]['content'] = f"{prompt} \n {data[k]}"
    all_messages.append(messages)

In [ ]:
all_messages[0]

[{'role': 'system', 'content': ''},
 {'role': 'user',
  'content': "Given a detailed description of a room's interior, identify all the distinct objects mentioned in the description. List these objects separated by commas, focusing solely on the items present in the room without including any descriptive or contextual information. For example, if the description mentions 'a cozy bedroom with a neatly made bed under a window, a dresser, and a potted plant', you should return 'bed, window, dresser, potted plant'. \n The room is a large, open, and clean living room with a white color scheme. It features a couch, a chair, and a television. The room also has a dining table and a few decorative items, such as a vase and a bottle. The overall atmosphere of the room is bright and inviting, with a focus on comfort and functionality."}]

In [ ]:
outputs = []
count = 0
batch_size = 16

In [ ]:
for out in pipe(
            all_messages,
            batch_size=batch_size,
            max_new_tokens=20,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95,
            stop_sequence="<|im_end|>",
):
    outputs.append(out)
    count += batch_size
    print(count)

In [ ]:
len(outputs)

232

In [ ]:
outputs[0]

[{'generated_text': [{'role': 'system', 'content': ''},
   {'role': 'user',
    'content': "Given a detailed description of a room's interior, identify all the distinct objects mentioned in the description. List these objects separated by commas, focusing solely on the items present in the room without including any descriptive or contextual information. For example, if the description mentions 'a cozy bedroom with a neatly made bed under a window, a dresser, and a potted plant', you should return 'bed, window, dresser, potted plant'. \n The room is a cozy living room with a white color scheme. It features a television on the wall, a couch, and a chair. There is also a dining table in the room. The living room is decorated with various books, which are scattered throughout the space. Additionally, there are two teddy bears in the room, one on the couch and the other on the floor. The room appears to be a comfortable and inviting space for relaxation and entertainment."},
   {'role': 'a

In [ ]:
outputs[1]

[{'generated_text': [{'role': 'system', 'content': ''},
   {'role': 'user',
    'content': "Given a detailed description of a room's interior, identify all the distinct objects mentioned in the description. List these objects separated by commas, focusing solely on the items present in the room without including any descriptive or contextual information. For example, if the description mentions 'a cozy bedroom with a neatly made bed under a window, a dresser, and a potted plant', you should return 'bed, window, dresser, potted plant'. \n The room is a cozy living room with a white color scheme. It features a television on the wall, a couch, and a chair. There is also a dining table in the room. The living room is decorated with various books, which are scattered throughout the space. Additionally, there are two teddy bears in the room, one on the couch and the other on the floor. The room appears to be a comfortable and inviting space for relaxation and entertainment."},
   {'role': 'a

In [ ]:
print()

couch, chair, television, dining table, books, teddy bear on couch, teddy bear on floor


"Given a detailed description of a room's interior, identify all the distinct objects mentioned in the description. List these objects separated by commas, focusing solely on the items present in the room without including any descriptive or contextual information. For example, if the description mentions 'a cozy bedroom with a neatly made bed under a window, a dresser, and a potted plant', you should return 'bed, window, dresser, potted plant'. \n The room is a cozy living room with a white color scheme. It features a television on the wall, a couch, and a chair. There is also a dining table in the room. The living room is decorated with various books, which are scattered throughout the space. Additionally, there are two teddy bears in the room, one on the couch and the other on the floor. The room appears to be a comfortable and inviting space for relaxation and entertainment."

In [ ]:
len(filenames), len(outputs)

(232, 232)

In [ ]:
object_caption = {}
for i in range(len(filenames)):
    object_caption[filenames[i]] = outputs[i][0]["generated_text"][-1]["content"]

In [ ]:
with open('main_objects_captions.json', 'w') as file:
    json.dump(object_caption, file, indent=4)